In [1]:
import os
import csv
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.python.keras.layers.core import Flatten

In [3]:
max_response_length = 500
vocab_size = 100000
vector_size = 10 # (?) not sure what a good vector size would be

In [4]:
data = []
os.path.join('C:\Users\Drew\Documents\app-school-450\data-sets\trip-test.csv', 'data-sets', 'trip-test.csv')
with open(os.path, newline = '', encoding = 'utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='|')
    for row in csv_reader:
        current_row = row[0]
        if len(current_row) > max_response_length:
            current_row = current_row[0:max_response_length]
        data.append(current_row)

TypeError: expected str, bytes or os.PathLike object, not module

In [ ]:
training_size = len(list(csv_reader)) / 2
training_set = data[0:training_size]
testing_set = data[training_size:]

In [ ]:
# only making the word index after the training set. 
# test set should not influence the word_index. whole point of testing.

tokenizer = Tokenizer(num_words = vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(training_set)
word_index = tokenizer.word_index
# word_index # prints the word index

In [ ]:
# still create squences for the training set.
# only make sequences for the test set. don't add testSet tokens to word_index
training_sequences = tokenizer.texts_to_sequences(training_set)
padded_training_sequences = pad_sequences(training_sequences, maxlen = max_response_length)
testing_sequences = tokenizer.texts_to_sequences(testing_set)
padded_testing_sequences = pad_sequences(testing_sequences, maxlen = max_response_length)

In [ ]:
# Convert sequence arrays to numPy arrays
padded_training_sequences = np.array(padded_training_sequences)
padded_testing_sequences = np.array(padded_testing_sequences)

In [ ]:
# Building the model
model = Sequential()
model.add(Embedding(vocab_size, vector_size, input_length=max_response_length, name="embedding"))
model.add(GlobalAveragePooling1D())
model.add(Flatten())
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
# Train the model
num_epochs = 30
model.fit(padded_training_sequences, training_set, epochs=num_epochs,
            validation_data=(padded_testing_sequences, testing_set), verbose=2)

In [ ]:
# We'll figure out when to add this in later.
with open('wordindex.txt', 'w') as file:
    file.writelines('{}'.format(word_index))